# 📚 Document Loading e Splitters - Preparando Dados para RAG

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-08_img_01.png)

**Módulo 8 de 17 - LangChain v0.3**

Fala pessoal! Tô aqui de novo com vocês pra mais um módulo do nosso curso de LangChain! 🚀

Tá, mas o que diabos é Document Loading e Splitters? Imagina que você tem uma biblioteca gigante e precisa encontrar uma informação específica. Você não vai ler todos os livros né? Você vai direto no índice, nas páginas certas!

É exatamente isso que fazemos com documentos em IA. Pegamos textos enormes e dividimos em pedacinhos menores que fazem sentido. É tipo picar uma pizza - cada pedaço tem que ter tudo que precisa pra ser saboroso! 🍕

**Por que isso é importante?**
- Nos próximos módulos vamos implementar RAG (Retrieval Augmented Generation)
- RAG precisa de documentos bem organizados e divididos
- É a base para sistemas de busca inteligente

Bora começar!

## 🛠️ Setup Inicial - Instalando as Dependências

Antes de começar, vamos instalar tudo que precisamos. É tipo preparar os ingredientes antes de cozinhar!

In [ ]:
# Instalando as bibliotecas necessárias
!pip install langchain langchain-community python-docx pypdf pymupdf beautifulsoup4 requests

# Imports básicos
import os
from langchain.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
import requests
from bs4 import BeautifulSoup

## 📖 O que são Document Loaders?

Tá, mas o que são esses Document Loaders? Simples!

Imagina que você tem documentos em vários formatos:
- PDF (tipo aquele manual chato da TV)
- TXT (arquivo simples de texto)
- Páginas web (sites, blogs)
- Word, Excel, PowerPoint

Cada formato tem sua "linguagem" própria. Os Document Loaders são tipo tradutores universais que pegam qualquer formato e transformam em algo que o LangChain entende!

### Estrutura de um Document no LangChain

Todo documento no LangChain tem duas partes principais:
- **page_content**: O texto em si
- **metadata**: Informações sobre o documento (título, fonte, página, etc.)

É tipo um envelope: tem a carta (conteúdo) e as informações do envelope (metadados)!

**Dica!** Sempre guarde metadados! Eles são super úteis para rastrear de onde veio cada pedaço de informação.

In [ ]:
# Vamos criar um documento simples para entender a estrutura
from langchain.schema import Document

# Criando um documento básico
documento_exemplo = Document(
    page_content="Este é o conteúdo do meu documento sobre IA. A inteligência artificial está revolucionando o mundo!",
    metadata={
        "titulo": "Introdução à IA",
        "autor": "Pedro Guth",
        "data": "2024-01-15",
        "categoria": "tecnologia"
    }
)

print("Conteúdo do documento:")
print(documento_exemplo.page_content)
print("\nMetadados:")
print(documento_exemplo.metadata)

## 📄 Carregando Diferentes Tipos de Documentos

Agora vamos ver como carregar documentos de diferentes fontes. É tipo ter chaves para abrir diferentes tipos de portas!

In [ ]:
# 1. Carregando arquivo de texto simples
# Primeiro vamos criar um arquivo de exemplo
texto_exemplo = """
LangChain é uma biblioteca incrível para desenvolvimento de aplicações com IA.
Ela facilita muito a integração com diferentes modelos de linguagem.
Nos módulos anteriores, aprendemos sobre ChatModels, Prompts e Chains.
Agora estamos preparando dados para implementar RAG nos próximos módulos.
O RAG (Retrieval Augmented Generation) é uma técnica poderosa que combina busca e geração.
"""

# Salvando o arquivo
with open("exemplo.txt", "w", encoding="utf-8") as f:
    f.write(texto_exemplo)

# Carregando com TextLoader
from langchain.document_loaders import TextLoader

loader_texto = TextLoader("exemplo.txt", encoding="utf-8")
documentos_texto = loader_texto.load()

print("Documento carregado do arquivo TXT:")
print(f"Número de documentos: {len(documentos_texto)}")
print(f"Conteúdo: {documentos_texto[0].page_content[:100]}...")
print(f"Metadados: {documentos_texto[0].metadata}")

In [ ]:
# 2. Carregando conteúdo de páginas web
from langchain.document_loaders import WebBaseLoader

# Carregando uma página web (vamos usar uma página simples)
try:
    loader_web = WebBaseLoader("https://httpbin.org/html")
    documentos_web = loader_web.load()
    
    print("\nDocumento carregado da web:")
    print(f"Número de documentos: {len(documentos_web)}")
    print(f"Conteúdo (primeiros 200 chars): {documentos_web[0].page_content[:200]}...")
    print(f"Metadados: {documentos_web[0].metadata}")
    
except Exception as e:
    print(f"Erro ao carregar página web: {e}")
    print("Sem problemas! Vamos criar um documento web simulado:")
    
    # Simulando conteúdo web
    documento_web_simulado = Document(
        page_content="Este é o conteúdo de uma página web sobre LangChain. A página contém informações sobre como usar document loaders para processar diferentes tipos de arquivo.",
        metadata={"source": "https://exemplo.com/langchain", "title": "Guia LangChain"}
    )
    print(f"Conteúdo simulado: {documento_web_simulado.page_content}")

## ✂️ Text Splitters - Dividindo para Conquistar

Agora vem a parte mais importante! Tá, mas por que dividir os textos?

Imagina que você tem um livro de 500 páginas e quer encontrar informação sobre "como fazer brigadeiro". Você não vai dar o livro inteiro para alguém ler né? Você vai direto no capítulo de doces!

É exatamente isso que os Text Splitters fazem:
- Dividem textos grandes em pedaços menores
- Mantêm o contexto (não cortam no meio de uma frase)
- Facilitam a busca e processamento

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-08_img_02.png)

### Por que isso é importante para RAG?

Lembra que nos próximos módulos vamos implementar RAG? Então:
- RAG precisa encontrar informações relevantes rapidamente
- Textos muito grandes são difíceis de processar
- Pedaços pequenos são mais fáceis de buscar e comparar

**Dica!** O tamanho ideal dos chunks (pedaços) depende do seu caso de uso. Para perguntas e respostas: 200-500 caracteres. Para análise profunda: 1000-2000 caracteres.

In [ ]:
# Vamos criar um texto mais longo para demonstrar os splitters
texto_longo = """
O LangChain é uma biblioteca poderosa que facilita o desenvolvimento de aplicações com IA. 
Ela oferece uma abstração elegante para trabalhar com diferentes modelos de linguagem.

Nos módulos anteriores, aprendemos sobre ChatModels que nos permitem conversar com IAs como GPT e Gemini.
Também vimos Prompt Templates que padronizam nossas conversas com a IA.
As Chains nos ajudam a criar fluxos complexos de processamento.

Agora estamos preparando dados para RAG (Retrieval Augmented Generation).
O RAG é uma técnica que combina busca em documentos com geração de texto.
É muito útil para criar assistentes que podem responder baseados em documentos específicos.

Para implementar RAG, precisamos:
1. Carregar documentos de diferentes fontes
2. Dividir os textos em pedaços menores
3. Criar embeddings (vamos ver no próximo módulo)
4. Armazenar em um vector store
5. Implementar a busca e geração

Os Document Loaders e Text Splitters são o primeiro passo desta jornada.
Eles preparam nossos dados para os próximos passos do processo RAG.
"""

# Criando um documento com este texto
documento_longo = Document(
    page_content=texto_longo,
    metadata={"fonte": "curso_langchain", "modulo": 8}
)

print(f"Texto original tem {len(texto_longo)} caracteres")
print(f"Vamos dividir este texto em pedaços menores!")

## 🔧 CharacterTextSplitter - O Divisor Simples

O CharacterTextSplitter é o mais básico. Ele divide o texto baseado em um caractere específico (tipo \n para quebra de linha).

É tipo cortar uma pizza com uma régua - funciona, mas não é o mais inteligente!

In [ ]:
# Usando CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# Configurando o splitter
splitter_simples = CharacterTextSplitter(
    chunk_size=300,      # Tamanho máximo de cada pedaço
    chunk_overlap=50,    # Sobreposição entre pedaços (importante para manter contexto!)
    separator="\n\n"      # Divide onde tem duas quebras de linha
)

# Dividindo o documento
chunks_simples = splitter_simples.split_documents([documento_longo])

print(f"Documento dividido em {len(chunks_simples)} pedaços:")
print()

# Mostrando cada pedaço
for i, chunk in enumerate(chunks_simples):
    print(f"--- Pedaço {i+1} ({len(chunk.page_content)} chars) ---")
    print(chunk.page_content[:200] + "..." if len(chunk.page_content) > 200 else chunk.page_content)
    print(f"Metadados: {chunk.metadata}")
    print()

## 🧠 RecursiveCharacterTextSplitter - O Inteligente

Agora sim! Este é o splitter mais usado e inteligente. Por quê?

Ele tenta dividir o texto seguindo uma hierarquia:
1. Primeiro tenta dividir por parágrafos (\n\n)
2. Se ainda estiver grande, divide por frases (.)
3. Se ainda estiver grande, divide por palavras ( )
4. Por último, divide por caracteres

É tipo um chef experiente cortando ingredientes - ele sabe onde cortar para não estragar o sabor!

**Dica!** Este é o splitter que você vai usar 90% das vezes. Ele é inteligente o suficiente para manter o contexto.

In [ ]:
# Usando RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configurando o splitter inteligente
splitter_inteligente = RecursiveCharacterTextSplitter(
    chunk_size=400,        # Tamanho máximo de cada pedaço
    chunk_overlap=100,     # Sobreposição entre pedaços
    length_function=len,   # Função para medir o tamanho
    separators=["\n\n", "\n", ".", " ", ""]  # Hierarquia de separadores
)

# Dividindo o documento
chunks_inteligentes = splitter_inteligente.split_documents([documento_longo])

print(f"Documento dividido em {len(chunks_inteligentes)} pedaços com RecursiveCharacterTextSplitter:")
print()

# Mostrando cada pedaço
for i, chunk in enumerate(chunks_inteligentes):
    print(f"--- Pedaço Inteligente {i+1} ({len(chunk.page_content)} chars) ---")
    print(chunk.page_content)
    print(f"Metadados: {chunk.metadata}")
    print("-" * 50)

## 📊 Visualizando a Diferença entre Splitters

Vamos criar um gráfico para visualizar como cada splitter se comporta. É sempre bom ver os dados!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Coletando dados dos splitters
tamanhos_simples = [len(chunk.page_content) for chunk in chunks_simples]
tamanhos_inteligentes = [len(chunk.page_content) for chunk in chunks_inteligentes]

# Criando o gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Comparação de tamanhos
ax1.bar(range(len(tamanhos_simples)), tamanhos_simples, 
        alpha=0.7, label='Character Splitter', color='skyblue')
ax1.bar(range(len(tamanhos_inteligentes)), tamanhos_inteligentes, 
        alpha=0.7, label='Recursive Splitter', color='lightcoral')
ax1.set_xlabel('Número do Chunk')
ax1.set_ylabel('Tamanho (caracteres)')
ax1.set_title('Comparação de Tamanhos dos Chunks')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Distribuição de tamanhos
ax2.hist(tamanhos_simples, bins=5, alpha=0.7, label='Character Splitter', color='skyblue')
ax2.hist(tamanhos_inteligentes, bins=5, alpha=0.7, label='Recursive Splitter', color='lightcoral')
ax2.set_xlabel('Tamanho do Chunk (caracteres)')
ax2.set_ylabel('Frequência')
ax2.set_title('Distribuição dos Tamanhos')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEstatísticas:")
print(f"Character Splitter: {len(chunks_simples)} chunks, tamanho médio: {np.mean(tamanhos_simples):.1f}")
print(f"Recursive Splitter: {len(chunks_inteligentes)} chunks, tamanho médio: {np.mean(tamanhos_inteligentes):.1f}")

## 🎯 Parâmetros Importantes dos Splitters

Tá, mas quais são os parâmetros mais importantes? Vamos entender cada um:

### chunk_size
- Tamanho máximo de cada pedaço
- **Pequeno (100-300)**: Boa precisão, mas pode perder contexto
- **Médio (300-800)**: Equilíbrio entre precisão e contexto ⭐
- **Grande (800+)**: Muito contexto, mas busca menos precisa

### chunk_overlap
- Quantos caracteres se repetem entre chunks
- **Importante**: Evita que informações sejam cortadas no meio
- **Recomendação**: 10-20% do chunk_size

### separators
- Define onde o texto pode ser cortado
- **Padrão**: ["\n\n", "\n", ".", " ", ""]
- **Para código**: ["\n\n", "\n", ";", "{", "}", " "]

**Dica!** Para RAG, comece com chunk_size=500 e chunk_overlap=100. Depois ajuste baseado nos seus testes!

In [ ]:
# Testando diferentes configurações
configurações = [
    {"nome": "Pequenos", "chunk_size": 200, "chunk_overlap": 50},
    {"nome": "Médios", "chunk_size": 400, "chunk_overlap": 80},
    {"nome": "Grandes", "chunk_size": 800, "chunk_overlap": 150}
]

resultados = []

for config in configurações:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    
    chunks = splitter.split_documents([documento_longo])
    
    resultado = {
        "nome": config["nome"],
        "num_chunks": len(chunks),
        "tamanho_medio": np.mean([len(chunk.page_content) for chunk in chunks]),
        "chunk_size_config": config["chunk_size"]
    }
    
    resultados.append(resultado)
    
    print(f"{config['nome']}: {len(chunks)} chunks, tamanho médio: {resultado['tamanho_medio']:.1f}")

print("\nQual configuração escolher?")
print("📍 Pequenos: Ótimo para busca precisa")
print("🎯 Médios: Equilibrio ideal para a maioria dos casos")
print("📚 Grandes: Bom quando você precisa de muito contexto")

## 🌐 Exemplo Prático - Processando Conteúdo Web

Vamos fazer um exemplo mais próximo da realidade: carregar conteúdo de uma página web e preparar para RAG!

In [ ]:
# Simulando conteúdo de um blog sobre IA
conteudo_blog = """
# Inteligência Artificial no Brasil: O Futuro é Agora

A inteligência artificial está transformando o mercado brasileiro de forma acelerada. 
Empresas de todos os tamanhos estão adotando soluções de IA para melhorar seus processos.

## Principais Aplicações

### Atendimento ao Cliente
Chatbots inteligentes estão revolucionando o atendimento. Eles conseguem resolver 80% dos problemas sem intervenção humana.
Empresas como Magazine Luiza e Bradesco já implementaram soluções avançadas.

### Análise de Dados
Machine Learning está sendo usado para analisar padrões de consumo e prever tendências.
O setor financeiro é pioneiro nesta área, usando IA para detectar fraudes e avaliar riscos.

### Automação de Processos
RPA (Robotic Process Automation) combinado com IA está automatizando tarefas repetitivas.
Isso libera funcionários para atividades mais estratégicas e criativas.

## Desafios e Oportunidades

O maior desafio é a capacitação profissional. O mercado demanda profissionais qualificados em IA.
Por outro lado, isso cria oportunidades enormes para quem se especializar na área.

## Conclusão

A IA não é mais coisa do futuro - é realidade do presente.
Empresas que não se adaptarem ficarão para trás na competição.
"""

# Criando documento simulando conteúdo web
documento_blog = Document(
    page_content=conteudo_blog,
    metadata={
        "source": "https://blog-ia-brasil.com/futuro-agora",
        "title": "IA no Brasil: O Futuro é Agora",
        "author": "Tech Blog Brasil",
        "date": "2024-01-15",
        "category": "tecnologia"
    }
)

print("Documento do blog carregado!")
print(f"Tamanho: {len(conteudo_blog)} caracteres")

In [ ]:
# Processando o conteúdo do blog para RAG
splitter_blog = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks_blog = splitter_blog.split_documents([documento_blog])

print(f"Blog dividido em {len(chunks_blog)} chunks para RAG:")
print("\n" + "="*60)

for i, chunk in enumerate(chunks_blog):
    print(f"\nChunk {i+1} ({len(chunk.page_content)} chars):")
    print("-" * 30)
    print(chunk.page_content.strip())
    print(f"Metadados: {chunk.metadata}")

print("\n" + "="*60)
print("Liiindo! Agora temos os dados prontos para o próximo módulo (Vector Store)!")

## 🔄 Fluxo Completo - Da Fonte ao Chunk

Vamos visualizar todo o processo que acabamos de aprender:

```mermaid
graph TD
    A[Documento Original] --> B[Document Loader]
    B --> C[Document Object]
    C --> D[Text Splitter]
    D --> E[Chunks Menores]
    E --> F[Prontos para Vector Store]
    
    B1[PDF] --> B
    B2[TXT] --> B
    B3[Web] --> B
    B4[Word] --> B
    
    D1[CharacterTextSplitter] --> D
    D2[RecursiveCharacterTextSplitter] --> D
```

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-08_img_03.png)

## 💡 Dicas Avançadas e Boas Práticas

Agora que você já sabe o básico, vou compartilhar algumas dicas de quem já quebrou a cabeça com isso:

### 1. Preservando Metadados Importantes
- Sempre mantenha informações de fonte
- Adicione timestamps quando relevante
- Use metadados para filtrar buscas

### 2. Tamanho dos Chunks para Diferentes Casos
- **FAQ/Q&A**: 100-300 caracteres
- **Documentação técnica**: 400-800 caracteres
- **Artigos longos**: 600-1200 caracteres
- **Código**: 200-500 caracteres

### 3. Overlap Estratégico
- Para textos técnicos: 20% do chunk_size
- Para narrativas: 15% do chunk_size
- Para listas/FAQ: 10% do chunk_size

**Dica!** No próximo módulo (Vector Store), vamos ver como esses chunks são transformados em embeddings para busca semântica. É aí que a mágica do RAG realmente acontece!

In [ ]:
# Exemplo de preservação de metadados estratégicos
def processar_documento_completo(conteudo, fonte, categoria):
    """Função para processar documento mantendo metadados importantes"""
    
    # Criando documento com metadados ricos
    documento = Document(
        page_content=conteudo,
        metadata={
            "fonte": fonte,
            "categoria": categoria,
            "tamanho_original": len(conteudo),
            "processado_em": "2024-01-15",
            "versao_langchain": "0.3"
        }
    )
    
    # Configurando splitter otimizado
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=80,
        separators=["\n\n", "\n", ".", "!", "?", ";", " ", ""]
    )
    
    # Dividindo e enriquecendo metadados
    chunks = splitter.split_documents([documento])
    
    # Adicionando número do chunk nos metadados
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk_id"] = i
        chunk.metadata["total_chunks"] = len(chunks)
        chunk.metadata["chunk_size"] = len(chunk.page_content)
    
    return chunks

# Testando a função
chunks_processados = processar_documento_completo(
    conteudo=conteudo_blog,
    fonte="blog_ia_brasil",
    categoria="tecnologia"
)

print(f"Processamento completo: {len(chunks_processados)} chunks gerados")
print(f"\nExemplo de metadados enriquecidos:")
print(chunks_processados[0].metadata)

## 🎯 Exercícios Práticos

Agora é sua vez de praticar! Bora colocar a mão na massa!

### Exercício 1: Processando Diferentes Tipos de Conteúdo

Crie uma função que processe diferentes tipos de documentos e encontre a configuração ideal de chunk para cada tipo.

In [ ]:
# EXERCÍCIO 1: Complete o código abaixo

# Documentos de exemplo de diferentes tipos
doc_faq = """
P: Como instalar o LangChain?
R: Use pip install langchain

P: O que é um Document Loader?
R: É uma ferramenta para carregar documentos de diferentes fontes

P: Qual a diferença entre splitters?
R: RecursiveCharacterTextSplitter é mais inteligente que CharacterTextSplitter
"""

doc_tecnico = """
A arquitetura do LangChain é baseada em componentes modulares que permitem flexibilidade máxima.
Os Document Loaders são responsáveis por converter diferentes formatos em objetos Document padronizados.
Este processo de normalização é fundamental para o funcionamento correto dos Text Splitters.
Os Text Splitters implementam algoritmos sofisticados para dividir texto mantendo contexto semântico.
"""

# TODO: Complete a função abaixo
def encontrar_melhor_config(documento, tipo_doc):
    """Encontra a melhor configuração de chunk para cada tipo de documento"""
    
    configs = {
        "faq": {"chunk_size": 150, "chunk_overlap": 20},
        "tecnico": {"chunk_size": 400, "chunk_overlap": 80}
    }
    
    # TODO: Implemente a lógica aqui
    # 1. Pegar a config do tipo de documento
    # 2. Criar o splitter
    # 3. Processar o documento
    # 4. Retornar os chunks
    
    config = configs.get(tipo_doc, {"chunk_size": 300, "chunk_overlap": 50})
    
    # Seu código aqui...
    
    return chunks  # Remova esta linha e implemente!

# Teste sua função
# chunks_faq = encontrar_melhor_config(doc_faq, "faq")
# chunks_tecnico = encontrar_melhor_config(doc_tecnico, "tecnico")

print("💡 Dica: Pense em como cada tipo de documento deve ser dividido!")
print("FAQ: pedaços pequenos, uma pergunta por chunk")
print("Técnico: pedaços maiores, mantendo contexto completo")

### Exercício 2: Análise de Qualidade dos Chunks

Crie uma função que analise a qualidade dos chunks gerados e sugira melhorias.

In [ ]:
# EXERCÍCIO 2: Complete a função de análise

def analisar_qualidade_chunks(chunks):
    """Analisa a qualidade dos chunks e sugere melhorias"""
    
    # TODO: Implemente as análises abaixo
    
    # 1. Calcular estatísticas básicas
    tamanhos = [len(chunk.page_content) for chunk in chunks]
    
    # 2. Verificar chunks muito pequenos (< 50 chars)
    chunks_pequenos = 0  # TODO: contar chunks pequenos
    
    # 3. Verificar chunks muito grandes (> 1000 chars)  
    chunks_grandes = 0   # TODO: contar chunks grandes
    
    # 4. Verificar chunks que terminam no meio de palavra
    chunks_mal_cortados = 0  # TODO: contar chunks mal cortados
    
    # Seu código aqui...
    
    # Relatório
    print("📊 Análise de Qualidade dos Chunks:")
    print(f"Total de chunks: {len(chunks)}")
    print(f"Tamanho médio: {np.mean(tamanhos):.1f} caracteres")
    print(f"Chunks muito pequenos: {chunks_pequenos}")
    print(f"Chunks muito grandes: {chunks_grandes}")
    print(f"Chunks mal cortados: {chunks_mal_cortados}")
    
    # TODO: Adicionar sugestões de melhoria
    print("\n💡 Sugestões:")
    if chunks_pequenos > 0:
        print(f"- Considere aumentar chunk_size (muitos chunks pequenos)")
    # Adicione mais sugestões...

# Teste com os chunks que criamos
print("Análise dos chunks do blog:")
analisar_qualidade_chunks(chunks_blog)

## 📈 Preparando para os Próximos Módulos

Liiindo! Você chegou até aqui e já sabe como:
- Carregar documentos de diferentes fontes
- Dividir textos de forma inteligente
- Configurar splitters para diferentes casos de uso
- Manter metadados importantes

### O que vem pela frente?

**Módulo 9 - Vector Store e Embeddings**:
- Vamos transformar nossos chunks em embeddings (vetores)
- Aprender sobre diferentes vector stores
- Implementar busca semântica

**Módulo 10 - RAG Implementation**:
- Juntar tudo: chunks + embeddings + LLM
- Criar um sistema de pergunta e resposta
- Otimizar a recuperação de informações

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/langchain-modulo-08_img_04.png)

**Dica!** Guarde bem os conceitos deste módulo! Eles são a base de tudo que vem pela frente no RAG.

## 🎉 Resumo do Módulo

Cara, que jornada incrível! Vamos recapitular o que aprendemos:

### 🔑 Conceitos Principais

1. **Document Loaders**: Carregam documentos de qualquer formato
2. **Text Splitters**: Dividem textos mantendo contexto
3. **Chunks**: Pedaços pequenos e contextualizados de texto
4. **Metadados**: Informações sobre os documentos

### 🛠️ Ferramentas que Dominamos

- `TextLoader`: Para arquivos de texto
- `WebBaseLoader`: Para páginas web
- `CharacterTextSplitter`: Divisão simples
- `RecursiveCharacterTextSplitter`: Divisão inteligente ⭐

### 📏 Configurações Importantes

- **chunk_size**: 300-500 para a maioria dos casos
- **chunk_overlap**: 10-20% do chunk_size
- **separators**: Use a hierarquia padrão

### 🚀 Próximos Passos

Agora que nossos dados estão prontos, vamos transformá-los em embeddings e implementar RAG!

**Exercício para casa**: Pense em um documento que você gostaria de processar (PDF, site, arquivo). No próximo módulo vamos transformar ele em um sistema de busca inteligente!

Nos vemos no Módulo 9! Bora para Vector Stores! 🚀

In [ ]:
# Código final - Limpeza dos arquivos criados
import os

# Removendo arquivo temporário
if os.path.exists("exemplo.txt"):
    os.remove("exemplo.txt")
    print("✅ Arquivo temporário removido")

print("\n🎯 Módulo 8 completo!")
print("📚 Você agora sabe preparar dados para RAG")
print("🚀 Próximo módulo: Vector Store e Embeddings")
print("\n" + "="*50)
print("Pedro Guth - LangChain v0.3 Course")
print("Módulo 8/17: Document Loading e Splitters")
print("="*50)